In [1]:
import acnportal
import sklearn

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from acnportal import acnsim
from acnportal import algorithms
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
import adacharge

In [2]:
def rmit_level_2_network(transformer_cap=80, evse_per_phase=10):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 400 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork()
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [3]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 400  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 7.2 # kW

In [5]:
level2_ev_fleet_network = rmit_level_2_network(transformer_cap=80, evse_per_phase=10)

64.2570281124498
115.94202898550725


In [6]:
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
              [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [7]:
# Events with 100 EVs per weekday
sessions_100 = get_synth_events(100)

c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [14]:
print(type(sessions_100))

<class 'acnportal.acnsim.events.event_queue.EventQueue'>


In [7]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    timezone = pytz.timezone('Australia/Melbourne')

    # Start and End times are used when collecting data.
    start = timezone.localize(datetime(2019, 6, 1))
    end = timezone.localize(datetime(2019, 7, 1))

    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    r['total_cost'] = r['energy_cost'] + r['demand_charge']
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    return r

In [8]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [9]:
cost_min_obj = [adacharge.ObjectiveComponent(adacharge.total_energy, 1000),
                adacharge.ObjectiveComponent(adacharge.tou_energy_cost),
                adacharge.ObjectiveComponent(adacharge.quick_charge, 1e-6),
                adacharge.ObjectiveComponent(adacharge.equal_share, 1e-12)
               ]
cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)

In [10]:
level2_80kW_untrl_100 = run_experiment(level2_ev_fleet_network, uncontrolled, deepcopy(sessions_100))

c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 645. Max violation is 50.33483854110497 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 646. Max violation is 78.70636198403577 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 647. Max violation is 78.70636198403577 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 648. Max violation is 78.70636198403577 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-package

In [11]:
level2_80kW_cost_min_100 = run_experiment(level2_ev_fleet_network, cost_min, deepcopy(sessions_100))

c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 5093. Max violation is 0.057961014492747154 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 5094. Max violation is 0.057961014492747154 A on Secondary B at time index 0.
  warnings.warn(


In [12]:
ev_100 = pd.DataFrame({
    'Level 2: Unctrl: 80 kW : 30 EVSEs':  level2_80kW_untrl_100,
    'Level 2: Min Cost: 80 kW : 30 EVSEs': level2_80kW_cost_min_100
})

In [13]:
ev_100.to_csv("results/100_EV_simulation.csv")

In [14]:
pd.read_csv("results/100_EV_simulation.csv", index_col=0)

,Level 2: Unctrl: 80 kW : 30 EVSEs,Level 2: Min Cost: 80 kW : 30 EVSEs
proportion_of_energy_delivered,0.642018,0.598849
energy_delivered,14967.027116,13960.656877
num_swaps,870.000000,870.000000
num_never_charged,468.000000,468.000000
energy_cost,2298.710607,1993.447905
demand_charge,3015.144000,1234.596000
total_cost,5313.854607,3228.043905
$/kWh,0.355037,0.231224
